このノートブックでは、Optiver Realized Volatility PredictionについてのEDA・日本語での内容理解を試みます。

参考にさせていただきました）
https://www.kaggle.com/chumajin/optiver-realized-eda-for-starter-version

基本的には自分のメモですが、コメント・ご指摘など大歓迎です！

up voteくださった方、ありがとうございます！

→　内容は随時更新していきます。

## コンペ概要（Overviewの翻訳など）

You will have hundreds of millions of rows of highly granular financial data at your fingertips, with which you'll design your model forecasting volatility over 10-minute periods.

→ ビッグデータから10分間のボラティリティを予測するモデルを設計します。

In this competition, you will be given 10 minutes of book data and we ask you to predict what the volatility will be in the following 10 minutes. Volatility will be measured as follows:

→ "predict what the volatility will be in the following 10 minutes."なので直前（10 min）のbookのデータは与えられているようです（自信がない）。

ボラティリティって？

→ 価格変動のようです。一年間の価格の対数値（log returns；後述）の標準偏差らしい。なお、urlはボラティリティの説明。

https://www.smbcnikko.co.jp/terms/japan/ho/J0280.html

チュートリアルノートブックがあるよ

→　こちら

https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data

評価方法は？

→ RMEPS:二乗平均平方根パーセント誤差

#### コンペのコード要件

* CPU Notebook <= 9 hours run-time
* GPU Notebook <= 9 hours run-time
* Internet 接続は×
* 外部データの利用可。学習済みモデルも使っていいよ。
* Submission file は submission.csv　の名前で提出

## データ

**テストセットの最初の数行のみをダウンロードできるコード競争です。**

→　提出ファイルは三行だけですが、実際はたくさんデータがあるようです（テストデータの最初の三行だけを読み込んでいる。）

以下はgoogle翻訳べた張り

### book_ [train / test] .parquet
区切られた寄木細工のファイルstock_id。市場に投入された最も競争力のある売買注文に関するオーダーブックデータを提供します。本の上位2つのレベルが共有されます。本の第1レベルは価格面でより競争力があり、第2レベルよりも実行が優先されます。

* ```stock_id``` -株式のIDコード。すべてのストックIDがすべてのタイムバケットに存在するわけではありません。Parquetは、ロード時にこの列をカテゴリデータ型に強制します。あなたはそれをint8に変換したいかもしれません。
* ```time_id``` -タイムバケットのIDコード。時間IDは必ずしも連続している必要はありませんが、すべての在庫で一貫しています。
* ```seconds_in_bucket``` -バケットの開始からの秒数。常に0から始まります。
* ```bid_price[1/2]``` -最も競争力のある購入レベルの正規化された価格。
* ```ask_price[1/2]``` -最も競争力のある販売レベルの正規化された価格。
* ```bid_size[1/2]``` -最も競争力のある購入レベルでの株式数。
* ```ask_size[1/2]``` -最も競争力のある/ 2番目に競争力のある販売レベルの株式数。

### trade_[train/test].parquet
区切られた寄木細工のファイルstock_id。実際に実行された取引に関するデータが含まれています。通常、市場では、実際の取引よりも受動的な売買意図の更新（本の更新）が多いため、このファイルは注文書よりもまばらであると予想される場合があります。

* ```stock_id``` - 同上。
* ```time_id``` - 同上。
* ```seconds_in_bucket```- 同上。トレードデータとブックデータは同じ時間枠から取得され、トレードデータは一般にまばらであるため、このフィールドは必ずしも0から始まるとは限らないことに注意してください。
* ```price``` -1秒間に発生する実行済みトランザクションの平均価格。価格は正規化されており、平均は各取引で取引された株式数によって加重されています。
* ```size``` -取引された株式の総数。
* ```order_count``` -発生している固有の取引注文の数。

### train.csv

* ```stock_id``` -上記と同じですが、これはcsvであるため、列はカテゴリではなく整数として読み込まれます。
* ```time_id``` - 同上。
* ```target``` -同じstock / time_idでの特徴データに続いて、10分間のウィンドウで計算された実現ボラティリティ。フィーチャデータとターゲットデータの間に重複はありません。詳細については、チュートリアルノートブックをご覧ください。

### test.csv

他のデータファイルと送信ファイルの間のマッピングを提供します。他のテストファイルと同様に、ほとんどのデータは、送信時にノートブックでのみ利用可能であり、最初の数行のみがダウンロード可能です。

* ```stock_id``` - 同上。
* ```time_id``` - 同上。
* ```row_id``` -送信行の一意の識別子。既存の時間ID /在庫IDのペアごとに1つの行があります。各時間枠には、必ずしもすべての個別の株式が含まれているわけではありません。

### sample_submission.csv
正しい形式のサンプル送信ファイル。

* ```row_id``` -test.csvと同じです。
* ```target``` -train.csvと同じ定義。ベンチマークは、train.csvのターゲット値の中央値を使用しています。

## コンテスト Q&A (抜粋)

https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/249752

同じstock_idは、すべての競合データセットの同じ株式を表していますか？

→　Yes

time_idについて

→　time_idはランダムにシャッフルされるため、異なるデータセット間のブリッジとして機能する以外の情報は含まれないことに注意してください；つまり、time_idは時系列データとはならないようです。

3行しかテストデータがないよ

→　コードをコミットすると、コードはバックグラウンドで設定された実際のテストに対して実行され、リーダーボードが更新されます。（裏にデータがある）

## 金融の基礎知識；キーワードなど

https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data

などを参考にしましています

#### WAP; Weighted averaged price

ノートブックを見ていると、重要な特徴量としてよく出てきます。こんな感じで実装されがち

```
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
```

オーダーブックから計算されます。日本語記事(Volume Weighted Average Price)はこちら

https://oneinvest.jp/vwap/

```
つまり、VWAP（その日の平均約定価格）を知ることができれば、VWAP価格で株を買えば少なくともその日に高値掴みすることはなくなり、平均的な価格で取引ができることになります。

```
とのことです。

#### Log returns

今回のコンペの目的が変動なので時間による差分（≒時間微分）をとることがリーズナブルな考え方ですが、ファイナンシャルでは株式価格の差よりも比率が重要となることが多いため、**差分**の対数をとってログで評価することが多いようです。をLog returnsとして表現しているみたいです。

→ 100ドルと102ドルとの差が2ドルと10ドルと12ドルとの差2ドルが同じわけないので、それぞれ2%と20%との差だって表現したい。log10をとるとlog(100)=2、log(102)=2.008。log(10)=1、log(12)=1.08なので、後者の方が差が大きくて、事象を上手く表現できそうですね、とのこと。

```
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

 df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
```

→.diff()を使っていることに注意しましょう！

#### Realized volatility

今回のコンペでは、直前（10 min）のbookのデータは与えられており、ターゲットとしているvolatilityに対して、bookのデータから計算されるrealized vovatilityを重要な指標として使ってくれ、と言っているみたいです。wapのlog returnの二乗平均和をとって計算されるようです。

```
book_example.loc[:,'log_return'] = log_return(book_example['wap'])

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
```

## データセットの確認

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

基本ライブラリの読み込み

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ざっくりとしたコンペの内容は以下の通り。

* 各銘柄（stock_id）で、特定の時間区切り（time_id）の10分間の時系列データ（seconds_in_bucket）があり、その時間区切りでの価格変動の大小についてボラティリティ(今回のtarget)として評価している。

* ~~ある銘柄（stock_id=0）の、とある時間区切り（time_id=4, 32, 34）のボラティリティを予測しよう。ただし、time_id=4は初めの少しだけしか時系列データがないし、time_id=32, 34にはデータがないよ~~

→ 上訂正：裏にデータを隠していて、今回作ったモデルでいろんなデータのボラティリティを測定するよ！

インプットデータの中身を確認します。

/optiver-realized-volatility-predictionに格納されています。今回のコンペで読み込むデータは以下の6つ。

* book_test.parquet
* book_train.parquet
* trade_test.parquet
* trade_train.parquet
* test.csv
* train.csv

trainとなるbook_test.parquet(trade_test.paruet)には'stock_id=0'のデータのみが格納されているのに対して、testとなるbook_train.parquet(trade_train.paruet)には種々のstock_idのデータが格納されています。なお、stock_idはある特定の銘柄のようです。

In [ ]:
! ls ../input/optiver-realized-volatility-prediction

print('***')
print('book_test.parquet')
! ls ../input/optiver-realized-volatility-prediction/book_test.parquet

print('***')
print('book_train.parquet')
! ls ../input/optiver-realized-volatility-prediction/book_train.parquet

stock_id=0について、bookのtestとtrainの中身を確認します。

基本的には同じデータが入っており、違いはtime_idのみのようです。あと、testデータはtime_id=4のデータで３行のseconds_in_bucketのみ（裏にテストデータが隠されている）。

In [ ]:
book_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0")
book_testparquet

In [ ]:
book_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
book_trainparquet

1000行までのデータを可視化すると、それぞれのtime_id（銘柄）でseconds_in_bucket（時刻）が純増している様子が見れます。

time_idごとにデータ数（行数）は違うようです（min73-max549）が、seconds_in_bucketは0から始まって600で終了するようです（1秒データが600で10分間）。

barplotと統計データを見ると、time_idごとのデータ数は平均で200くらいのようです。

In [ ]:
plt.plot( np.array( book_trainparquet['time_id'][:1000] ) )
plt.show()
plt.plot( np.array( book_trainparquet['seconds_in_bucket'][:1000] ) )
plt.show()

In [ ]:
print( book_trainparquet['time_id'].value_counts() )
print()
print( book_trainparquet['time_id'].value_counts().describe() )

sns.countplot( book_trainparquet['time_id'] )
ax = plt.gca()
ax.axes.xaxis.set_ticks([])
plt.show()

time_idが5のデータ（銘柄）について、推移を確認します。seconds_in_bucketを時刻として採用しています。

In [ ]:
book_trainparquet[ book_trainparquet['time_id']==5 ]

In [ ]:
book_data = book_trainparquet[ book_trainparquet['time_id']==5 ]

plt.figure(figsize=(20,6))
for i, col in enumerate(['bid_price1','ask_price1','bid_price2', 'ask_price2', ]):
    plt.plot(book_data["seconds_in_bucket"],book_data[col],'x-',label=col)

plt.legend()
plt.show()

plt.figure(figsize=(20,6))
for i, col in enumerate(['bid_size1','ask_size1','bid_size2', 'ask_size2', ]):
    plt.plot(book_data["seconds_in_bucket"],book_data[col],label=col)

plt.legend()
plt.show()

bookに続いてtradeのデータについても確認します。

In [ ]:
trade_testparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0")
trade_testparquet

In [ ]:
trade_trainparquet = pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_trainparquet

bookと同じく1000行までのデータを可視化すると、time_id,seconds_in_bucketの推移は似てはいますが、tradeのデータは急峻であることがわかります。

time_idごとのデータ数は平均で30(min2-max120)くらいのようです。すなわち、tradeのデータは非常にスパース

In [ ]:
plt.plot( np.array( trade_trainparquet['time_id'][:1000] ) )
plt.show()
plt.plot( np.array( trade_trainparquet['seconds_in_bucket'][:1000] ) )
plt.show()

In [ ]:
print( trade_trainparquet['time_id'].value_counts() )
print()
print( trade_trainparquet['time_id'].value_counts().describe() )

sns.countplot( trade_trainparquet['time_id'] )
ax = plt.gca()
ax.axes.xaxis.set_ticks([])
plt.show()

In [ ]:
trade_trainparquet[ trade_trainparquet['time_id']==5 ]

trade_dataに関してtime_idが5のデータ（銘柄）について、推移を確認します。seconds_in_bucketを時刻として採用しています。

合わせて、book_dataとtrade_dataを合わせて可視化します。trade_dataの'price'がbookデータのbid,askの範囲内で変動していることがわかります。

In [ ]:
trade_data = trade_trainparquet[ trade_trainparquet['time_id']==5 ]

plt.figure(figsize=(20,6))
for i, col in enumerate(['price',]):
    plt.plot(trade_data["seconds_in_bucket"],trade_data[col],'x-',label=col)

plt.legend()
plt.show()

plt.figure(figsize=(20,6))
for i, col in enumerate(['size','order_count',]):
    plt.plot(trade_data["seconds_in_bucket"],trade_data[col],label=col)

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,6))
for i, col in enumerate(['bid_price1','ask_price1','bid_price2', 'ask_price2', ]):
    plt.plot(book_data["seconds_in_bucket"],book_data[col],'x-',label=col)

plt.plot(trade_data["seconds_in_bucket"],trade_data['price'],'o--',lw=3,label=col)

plt.legend()
plt.show()

今回の目的変数となるtarget（＝ボラティリティ）についてです。

ボラティリティは価格変動の度合いのようです。

https://www.smbcnikko.co.jp/terms/japan/ho/J0280.html

~~testデータを読み込みます。今回のコンペではここの各行についてtargetを計算せよとのこと（つまり、submissionのデータは三行）~~

~~ここで振り返ると、stock_id=0に関しては、time_id=4のデータ（bookもtradeも）しかないので、time_id=32, 34の予測には工夫が必要なようです。~~

In [ ]:
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test

In [ ]:
book_testparquet

In [ ]:
trade_testparquet

trainに関しては、targetはstock_id（銘柄）ごとにtime_id, targetが設定されています。

In [ ]:
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
train

trainのターゲットの値を分析します。統計量、分布については以下。

In [ ]:
print( train['target'].describe() )
#
sns.displot(train['target'])
plt.show()

始めの10000行データを出力しました。trainのデータはかなり密に入っていることがわかります。

In [ ]:
plt.plot( train[:10000]['stock_id'] )
plt.show()
#
plt.plot( train[:10000]['time_id'] )
plt.show()
#
plt.plot( train[:10000]['target'] )
plt.show()

In [ ]:
train[train['stock_id']==1]

stock_id=0,1,2の三銘柄について、time_idでの推移（便宜的に）を見ますと、なんとなく関連性があり、同じタイミング（time_id）でtargetが大きくなっているようにも見受けられます。

~~このあたりの関係性をもって、stock_id=0のtime_id=32, 34のtargetを予測することになりそうです。ただし、time_id=32, 34の直接的なデータはないようです。近いところだと、time_id=31と62。~~

→ 提供されているデータは実際のテストデータとは違うため、この辺は気にしなくてよさそう。

In [ ]:
for tt in ([ train[ train['stock_id']==0 ], 
             train[ train['stock_id']==1 ],
             train[ train['stock_id']==2 ],]):
    plt.plot( tt[:100]['time_id'],tt[:100]['target'] )
    plt.xlabel('time_id')
    plt.ylabel('target')
plt.show()

## 重要な変数の確認

### WAP
ブックデータから計算されます。

wapもbookデータのbid,askの範囲内で変動していることがわかります。

In [ ]:
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

In [ ]:
wap = calc_wap(book_data)

plt.figure(figsize=(20,6))
for i, col in enumerate(['bid_price1','ask_price1','bid_price2', 'ask_price2', ]):
    plt.plot(book_data["seconds_in_bucket"],book_data[col],'x-',label=col)

plt.plot(book_data["seconds_in_bucket"],wap,'o--',lw=3,label='WAP')
plt.legend()
plt.show()

標準偏差っぽいけど、少し裾が広い

In [ ]:
sns.histplot( wap, kde=True, )

#### log return

wap→wapのlog→wapのdiff→log return（wapのlogのdiff）の順で可視化しています。

→np.log(wap).diff()とwap.diff()が変わらないけど、そんなもんかな（放置）。

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
plt.figure(figsize=(10,6))
plt.plot( book_data["seconds_in_bucket"], wap )
plt.show()

plt.figure(figsize=(10,6))
plt.plot( book_data["seconds_in_bucket"], np.log(wap) )
plt.show()

plt.figure(figsize=(10,6))
plt.plot( book_data["seconds_in_bucket"], wap.diff() )
plt.plot( book_data["seconds_in_bucket"], log_return(wap), 'x' )

#### log_returnの分析

In [ ]:
log_return_wap = log_return(wap)[1:]
#
sns.histplot( log_return_wap, kde=True, stat = 'probability' )
plt.show()
sns.histplot( log_return_wap, kde=True, stat = 'probability', cumulative = True )
plt.show()
# 絶対値
sns.histplot( np.abs(log_return_wap), kde=True, stat = 'probability' )

#### realized_volatility

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
realized_vol = realized_volatility(log_return(wap))
realized_vol

In [ ]:
train_list = [ 0,
               31,
               18, #ボラティリティの大きな銘柄
               43, #ボラティリティの小さな銘柄
             ]

book_trainparquet_list  = []

for i in train_list:
    book_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id="+str(i)))

In [ ]:
plt.figure(figsize=(10,20))

tid = 5

for id, book in enumerate(book_trainparquet_list):
    #
    book_tid = book[ book['time_id'] == tid ]
    wap = calc_wap( book_tid ) 
    #
    plt.subplot(3,1,1)
    plt.plot( book_tid['seconds_in_bucket'],wap, label='id='+str(train_list[id]) )
    plt.legend()

    logreturn = log_return(wap)
    plt.subplot(3,1,2)
    plt.plot(book_tid['seconds_in_bucket'], logreturn, 'x')

    plt.subplot(3,1,3)
    #sns.histplot(logreturn, stat = 'probability', kde=True,  )
    sns.distplot(logreturn, )
    
    realized_vol = realized_volatility(logreturn)
    print('id=', train_list[id])
    print(realized_vol)

plt.show()

## 時系列データ分析

https://www.kaggle.com/bextuychiev/how-to-detect-white-noise-and-random-walks-in-ts

に詳しく記載ありです。

日本語はこちらも参考にしました。

https://dajiro.com/entry/2020/05/02/191334

In [ ]:
wap = calc_wap(book_data)
log_return_wap = log_return(wap)[1:]
log_return_wap = log_return_wap[ log_return_wap!=0 ]

for ff in [wap,log_return_wap]:
    plt.figure(figsize=(4,4))
    plt.plot(ff[:-1],ff[1:],'o')
    plt.show()

statsmodelsでコレログラムの作成

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# 自己相関
fig = plot_acf(wap, lags=50)
plt.show()

# 偏自己相関
fig = plot_pacf(wap, lags=50)
plt.show()

https://qiita.com/eg_i_eg/items/94ea41edc96cbb3dca6a

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA

import warnings
warnings.filterwarnings("ignore") # warnings を表示させないようにする

# 次数の推定
print(sm.tsa.arma_order_select_ic(wap, max_ar=5, max_ma=5, ic='aic'))
print(sm.tsa.arma_order_select_ic(wap, max_ar=5, max_ma=5, ic='bic'))

Dickey-Fulluer(ADF)検定は定常性の検定として利用できる。

帰無仮説は「データ系列に単位根が存在する」

p値が大きな場合、単位根過程であるという帰無仮説を棄却できない→単位根を有する。

p値が小さい場合、単位根過程であるという帰無仮説を棄却できる→単位根を有していない＝定常性を満たしている。

https://tkstock.site/2021/05/15/python-%E6%99%82%E7%B3%BB%E5%88%97-%E5%AE%9A%E5%B8%B8%E6%80%A7-%E5%8D%98%E4%BD%8D%E6%A0%B9%E6%A4%9C%E5%AE%9A-adf%E6%A4%9C%E5%AE%9A-%E5%AE%9F%E8%A3%85/




In [ ]:
from statsmodels.tsa.stattools import adfuller

def analysis_adf(series):
    results = adfuller(series)

    print(f"ADF Statistic: {results[0]}")
    print(f"p-value: {results[1]}")
    print("Critical Values:")
    for key, value in results[4].items():
        print("\t%s: %.3f" % (key, value))

analysis_adf(wap)

In [ ]:
for id, book in enumerate(book_trainparquet_list):
    #
    book_tid = book[ book['time_id'] == tid ]
    tmp_wap  = calc_wap( book_tid )
    analysis_adf( tmp_wap )
    plt.plot(tmp_wap)
    plt.show()
    # 自己相関
    fig = plot_acf(tmp_wap, lags=50)
    plt.show()
    # 偏自己相関
    fig = plot_pacf(tmp_wap, lags=50)
    plt.show()

pandasでのコレログラム作成

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(wap)

In [ ]:
for id, book in enumerate(book_trainparquet_list):
    #
    book_tid = book[ book['time_id'] == tid ]
    wap = calc_wap( book_tid )
    autocorrelation_plot(wap)

## EDA（簡易）; stock_idでの分析

stock_idごとにtargetの分布に差異があるようです。 規格化すればおよそ同じには見える、、、？


In [ ]:
from sklearn.preprocessing import PowerTransformer

for id in [1,2,10,30,100]:
    #
    xx = np.array(train[train['stock_id']==id]['target'])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
    plt.xlim([0,0.04])
    sns.distplot( xx,ax=ax1 )
    #
    plt.xlim([-3,3])
    scaler = PowerTransformer()
    sns.distplot( scaler.fit_transform(xx.reshape(-1,1)), ax=ax2)
    #
    plt.show()

stock_idごとにtargetの統計量を算出します。

In [ ]:
stock = train.groupby("stock_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
stock

In [ ]:
stock.describe()

In [ ]:
for i in ['mean','median','std']:
    sns.displot(stock[i])
    plt.show()

一応、銘柄ごとにボラティリティの大きさに違いがあるようなので、銘柄の統計量情報だけでも少しだけ予測できそう。

→ 銘柄のボラティリティの平均値（'mean'）が大きいほどボラティリティが大きい傾向にある（ほんの少し）。

In [ ]:
train_info = train.copy()
train_info['mean']    = train['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
train_info['median']  = train['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
train_info['std']     = train['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))
train_info

https://pythondatascience.plavox.info/seaborn/%E6%95%A3%E5%B8%83%E5%9B%B3%E3%83%BB%E5%9B%9E%E5%B8%B0%E3%83%A2%E3%83%87%E3%83%AB

In [ ]:
from sklearn.metrics import r2_score
#
for col in ['mean','median','std']:
    #
    xx = train_info[col]
    yy = train_info['target']
    #
    print(col)
    print('r2 score:',r2_score(xx,yy))
    #
    plt.plot(xx,yy,'x')
    #
    coef = np.polyfit(xx,yy,1)
    y_pred = coef[0]*xx+ coef[1]
    print('fit 1d:',coef[0],'x+',coef[1])
    #
    plt.plot(y_pred,xx,'k-')
    #
    plt.show()

In [ ]:
stock

stockごとに統計量がどういった形をしているかを改めて確認

In [ ]:
plt.plot( np.sort( np.array( stock['mean'] ) ),'x' )
plt.show()
plt.plot( np.sort( np.array( stock['median'] ) ),'x' )
plt.show()
plt.plot( np.sort( np.array( stock['std'] ) ),'x' )
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(stock['mean'],stock['median'],'o')
plt.show()

plt.figure(figsize=(5,5))
plt.plot(stock['mean'],stock['std'],'o')
plt.show()

plt.figure(figsize=(5,5))
plt.plot(stock['median'],stock['std'],'o')
plt.show()

In [ ]:
trade_trainparquet

stock_idごとにvoalitityとprice,size,order_countの関係の確認

In [ ]:
import glob

ld = glob.glob('../input/optiver-realized-volatility-prediction/book_train.parquet/*')

list_price        = []
list_size         = []
list_order_count  = []

for i in stock['stock_id']:
    list_price.append( np.mean( pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i))['price'] ))
    list_size.append( np.mean( pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i))['size'] ))
    list_order_count.append( np.mean( pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i))['order_count'] ))

In [ ]:
#
labels=[ 'price', 'size', 'order_count']
for i, l in enumerate( [list_price, list_size, list_order_count] ):
    #
    plt.figure(figsize=(5,5))
    plt.plot(stock['mean'],l,'o')
    plt.ylabel(labels[i])
    plt.xlabel('mean volatility')
    plt.show()

In [ ]:
pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i))

In [ ]:
from sklearn.cluster import KMeans

nc = 5

cust_array = stock.iloc[:,1:4]
labels = KMeans(n_clusters=nc).fit_predict(cust_array)

plt.figure(figsize=(5,5))
for i in range(nc):
    xx = stock['mean'][labels==i]
    yy = stock['std'][labels==i]
    plt.plot(xx,yy,'o')
plt.show()

## 簡易モデルの作成（線形回帰）


おためしで、統計量（median）から回帰

→なお、スコアは全然ダメダメです。

In [ ]:
xcol = 'median'
y = np.array( train_info['target'] )
X = np.array( train_info[xcol] ).reshape(-1,1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

scaler = StandardScaler()
#scaler = PowerTransformer()
scaler.fit(X)

clf = LinearRegression()
#clf = Lasso()
#clf = SVR()

clf.fit(scaler.transform(X),y)

y_pred = clf.predict(scaler.transform(X))

plt.figure(figsize=(4,4))

plt.plot(y,y_pred,'x')

min = np.min( np.array( np.min(y), np.min(y_pred) ) )
max = np.max( np.array( np.max(y), np.max(y_pred) ) )
#
plt.plot( [0.9*min,1.1*max],[0.9*min,1.1*max], 'k-' )

plt.xlabel('Actual')
plt.ylabel('Estimated')

print(r2_score(y,y_pred))

plt.show()

## 一旦ここで提出(Score 0.98)

提出条件の確認のため、上の雑なモデルで一旦Submissionしてみます。testデータから作成します。stock_idを読み込んで、対応するstock_idのmeanを読み取り、そこから回帰式で予測値を返す簡易プログラムになっています。

これでも無事、提出されるようです；スコアはさんざん（0.98）ですが、、、

なお、下手に回帰するよりも'median'をそのまま入れる方がスコアが高いよう（0.68）です。→targetを差分にした方が扱いやすい？？

！注意 インターネット接続は切ること

In [ ]:
test

In [ ]:
X_test = np.array( test['stock_id'].map(dict(zip(stock['stock_id'], stock[xcol]))) ).reshape(-1,1)
X_test

In [ ]:
#y_pred = X_test
y_pred = clf.predict(scaler.transform(X_test))
y_pred

In [ ]:
sub = test.copy()
sub['target'] = y_pred
sub = sub.drop("stock_id",axis=1)
sub = sub.drop("time_id",axis=1)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)

## スコア

今回のスコアはrmspeです。ので、準備。

In [ ]:
#
def rmspe(y_true, y_pred):
    #
    yt = np.array(y_true)
    yp = np.array(y_pred)
    
    res = np.square((yt-yp)/yt)
    res = np.mean(res)
    res = np.sqrt(res)
    
    return res
    
    #return (np.sqrt(np.mean(np.square((y_true-y_pred)/y_true))))
#
#print(rmspe(y,y_pred))

## EDA（簡易）; time_idでの分析

time_idごとにボラティリティの違いがありそうですね（つまり、相場が大きく変動するタイミングは銘柄全体で変動が大きい）

→ time_id（の統計量）を説明変数に入れる意味はありそう。

In [ ]:
train['time_id'].describe()

In [ ]:
fig, ax = plt.subplots(1,5,figsize=(20,4))

for i, id in enumerate([5,11,16,31,62]):
    #
    #plt.xlim([0,0.05])
    sns.distplot( train[train['time_id']==id]['target'],ax=ax[i] )

plt.show()

In [ ]:
time = train.groupby("time_id")["target"].agg(["mean","median","std","count","sum"]).reset_index()
time

In [ ]:
time.describe()

In [ ]:
for i in ['mean','median','std']:
    sns.displot(time[i])
    plt.show()

In [ ]:
train_info = train.copy()
train_info['mean']    = train['time_id'].map(dict(zip(time['time_id'], time['mean'])))
train_info['median']  = train['time_id'].map(dict(zip(time['time_id'], time['median'])))
train_info['std']     = train['time_id'].map(dict(zip(time['time_id'], time['std'])))
train_info

In [ ]:
from sklearn.metrics import r2_score
#
for col in ['mean','median','std']:
    #
    xx = train_info[col]
    yy = train_info['target']
    #
    print(col)
    print('r2 score:',r2_score(xx,yy))
    #
    plt.plot(xx,yy,'x')
    #
    coef = np.polyfit(xx,yy,1)
    y_pred = coef[0]*xx+ coef[1]
    print('fit 1d:',coef[0],'x+',coef[1])
    #
    plt.plot(y_pred,xx,'k-')
    #
    plt.show()

## EDA; book/tradeデータ

各銘柄（stock_idの推移を比較します）

In [ ]:
train_list = [ 0,
               31,
               stock['stock_id'][stock['mean'].idxmax()], #ボラティリティの大きな銘柄
               stock['stock_id'][stock['mean'].idxmin()], #ボラティリティの小さな銘柄
             ]

book_trainparquet_list  = []
trade_trainparquet_list = []

for i in train_list:
    book_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id="+str(i)))
    trade_trainparquet_list.append(pd.read_parquet("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id="+str(i)))

train_list

In [ ]:
for t, tt in enumerate( [ 5, time['time_id'][time['mean'].idxmin()], time['time_id'][time['mean'].idxmax()] ] ):
    #
    plt.figure(figsize=(20,5))
    #
    print('time_id == ', tt )
    #
    for i, trade_t in enumerate(trade_trainparquet_list):
        #
        trade_data = trade_t[ trade_t['time_id']==tt ]
        plt.plot(trade_data["seconds_in_bucket"],trade_data['price'],'o--',label='stock_id='+str(train_list[i]))

    plt.legend()
    plt.show()

時系列データをヒートマップとして可視化

暫定で stock_id=1, time_id=5で規格化

In [ ]:
from sklearn.preprocessing import StandardScaler

i = 3
trade_data = trade_trainparquet_list[i][ trade_trainparquet_list[i]['time_id']==5 ]
scaler = StandardScaler()

scaler.fit( trade_data.iloc[:,2:] )

In [ ]:
for i, trade_t in enumerate(trade_trainparquet_list):
    #
    print('stock_id='+str(train_list[i]))
          
    trade_data = trade_t[ trade_t['time_id']==5 ]
    #
    td = scaler.transform( trade_data.iloc[:,2:] )
    #
    heat = np.zeros([3,600])
    heat[:,:] = np.nan
    #
    for i, sec in enumerate(trade_data['seconds_in_bucket']):
        heat[0,sec] = td[i,0]
        heat[1,sec] = td[i,1]
        heat[2,sec] = td[i,2]
    #
    heat = pd.DataFrame(heat.T)
    heat = heat.interpolate()
    #
    plt.figure(figsize=(20,3))
    sns.heatmap(heat.T)
    plt.show()

## モデル(簡易；XGBoost)の構築と提出(Score 0.52)

stock_id, time_idの統計量を使ってtargetの回帰を行います。

なお、xgboostが一番成績が良かったので、こいつで提出したらPublic Scoreは0.52071でした。

In [ ]:
train_info = train.copy()
train_info['stock_id_mean']     = train['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
train_info['stock_id_median']   = train['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
train_info['stock_id_std']      = train['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))
train_info['time_id_mean']      = train['time_id'].map(dict(zip(time['time_id'], time['mean'])))
train_info['time_id_median']    = train['time_id'].map(dict(zip(time['time_id'], time['median'])))
train_info['time_id_std']        = train['time_id'].map(dict(zip(time['time_id'], time['std'])))
train_info

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=12345678, shuffle=True)

In [ ]:
y = np.array( train['target'] )
X = np.array( train_info.iloc[:,3:] )

LinearRegression,Lasso,Ridge,SVR,XGBRegressorを比較

XGBRegressorの成績が良い（ハイパーパラメータチューニングはしていない）。

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

scaler = StandardScaler()
#scaler = PowerTransformer()
scaler.fit(X)

models = [LinearRegression(),Lasso(),Ridge(),SVR(),xgb.XGBRegressor()]
scores = []

for clf in models:

    scores = []
    
    print(clf)

    plt.figure(figsize=(4,4))    
    
    for train_index, test_index in kf.split(X, y):
        #
        X_train = scaler.transform( X[train_index] )
        y_train = y[train_index]
        #
        X_test  = scaler.transform( X[test_index] )
        y_test  = y[test_index]
        #
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        #
        plt.plot(y_pred,y_test,'x')
        #
        scores.append((rmspe(y_test,y_pred)))
    #
    print(np.mean(np.array(scores)))
    #
    min = np.min( np.array( np.min(y_test), np.min(y_pred) ) )
    max = np.max( np.array( np.max(y_test), np.max(y_pred) ) )
    #
    plt.plot( [0.9*min,1.1*max],[0.9*min,1.1*max], 'k-' )    
    #
    plt.show()

XGBRegressorでSubmissionを作成します。

In [ ]:
test_info = test.copy()
test_info['stock_id_mean']     = test['stock_id'].map(dict(zip(stock['stock_id'], stock['mean'])))
test_info['stock_id_median']   = test['stock_id'].map(dict(zip(stock['stock_id'], stock['median'])))
test_info['stock_id_std']      = test['stock_id'].map(dict(zip(stock['stock_id'], stock['std'])))
test_info['time_id_mean']      = test['time_id'].map(dict(zip(time['time_id'], time['mean'])))
test_info['time_id_median']    = test['time_id'].map(dict(zip(time['time_id'], time['median'])))
test_info['time_id_std']       = test['time_id'].map(dict(zip(time['time_id'], time['std'])))
test_info

In [ ]:
clf  = models[4]
print(clf)

In [ ]:
X_test = np.array( test_info.iloc[:,3:] )

clf.fit(scaler.transform(X),y)

y_pred =clf.predict(scaler.transform(X_test))
y_pred

sub = test.copy()
sub['target'] = y_pred
sub = sub.drop("stock_id",axis=1)
sub = sub.drop("time_id",axis=1)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)

## モデルANN(WIP)

MLBコンペでANNが出てきていたので、勉強がてら使ってみます。

参考にしています。

https://www.kaggle.com/ulrich07/mlb-ann-with-lags-tf-keras

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

import gc

In [ ]:
def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    nh = 50
    x = L.Dense(nh, activation="relu", name="d1")(inp)
    x = L.Dense(nh, activation="relu", name="d2")(x)
    #x = L.Dense(nh, activation="relu", name="d3")(x)
    #preds = L.Dense(4, activation="linear", name="preds")(x)
    preds = L.Dense(1, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model

In [ ]:
net = make_model(X.shape[1])
print(net.summary())

In [ ]:
nets = []
EPOCHS  = 10

y_tests = []
y_preds = []

idx = 0
for tr_idx, val_idx in kf.split(X, y):
    #
    idx + 1
    print("FOLD:", idx)
    #tr_idx, val_idx = folds[idx]
    ckpt = ModelCheckpoint(f"w{idx}.h5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.0005)
    es = EarlyStopping(monitor='val_loss', patience=6)
    reg = make_model(X.shape[1])
    reg.fit(X[tr_idx], y[tr_idx], epochs=EPOCHS, batch_size=30_000, 
            validation_data=(X[val_idx], y[val_idx]),
            verbose=1, callbacks=[ckpt, reduce_lr, es])
    reg.load_weights(f"w{idx}.h5")
    
    y_preds.append(reg.predict(X[val_idx]))
    y_tests.append(y[val_idx]) 
    
    nets.append(reg)
    gc.collect()

スコアはメモリ足りなくなっているので保留

In [ ]:
plt.figure(figsize=(4,4)) 
for y_test, y_pred in zip( y_tests, y_preds ):
    plt.plot(y_test,y_pred,'x')

min = np.min( np.array( np.min(y_test), np.min(y_pred) ) )
max = np.max( np.array( np.max(y_test), np.max(y_pred) ) )
#
plt.plot( [0.9*min,1.1*max],[0.9*min,1.1*max], 'k-' )   
plt.show()

## モデル（ハイパーパラメータチューニング）の構築と提出

optunaを使ってチューニングします。下敷きはこちら

https://www.kaggle.com/matsuosan/japanese-xgb-lgbm-voting-optuna-boston

あんまりうまくいっていないので、参考にならないです。

In [ ]:
import optuna

In [ ]:
y = np.array( train['target'] )
X = np.array( train_info.iloc[:,3:] )

In [ ]:
train_info

In [ ]:
test_info

xgboostの最適化を選択します。

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

scaler = StandardScaler()
#scaler = PowerTransformer()
scaler.fit(X)

```
<bound method XGBModel.get_params of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)>
```

In [ ]:
import xgboost as xgb

def objective_xgb(trial):
    #
    if trial.number == 0 :
        learning_rate = trial.suggest_loguniform('learning_rate', 0.3, 0.3)
        gamma         = trial.suggest_loguniform('gamma', 1e-8, 1e-8)
        max_depth     = trial.suggest_int('max_depth', 6, 6)
        min_child_weight = trial.suggest_loguniform('min_child_weight', 1.0, 1.0)
        #max_delta_step   = trial.suggest_uniform('max_delta_step', 1e-10, 1e-10)
        subsample  = trial.suggest_uniform('subsample', 1.0, 1.0)
        reg_lambda = trial.suggest_uniform('reg_lambda', 1.0, 1.0)
        reg_alpha  = trial.suggest_uniform('reg_alpha', 0.0, 0.0)
    else:
        learning_rate = trial.suggest_loguniform('learning_rate', 1e-8, 1.0)
        gamma         = trial.suggest_loguniform('gamma', 1e-15, 1e-5)
        max_depth     = trial.suggest_int('max_depth', 1, 20)
        min_child_weight = trial.suggest_loguniform('min_child_weight', 1e-8, 1e3)
        #max_delta_step   = trial.suggest_uniform('max_delta_step', 0, 1.0)
        subsample  = trial.suggest_uniform('subsample', 0.0, 1.0)
        reg_lambda = trial.suggest_uniform('reg_lambda', 0.0, 1000.0)
        reg_alpha  = trial.suggest_uniform('reg_alpha', 0.0, 1000.0)
        #reg_alpha  = trial.suggest_loguniform('reg_alpha', 1e-15, 1e4)
    #
    
    clf = xgb.XGBRegressor(
                            learning_rate = learning_rate, 
                            subsample = subsample,
                            max_depth = max_depth,
                            min_child_weight = min_child_weight, 
                            max_delta_step = 0,  # 1e-10で発散したため、0で固定
                            reg_lambda = reg_lambda,
                            gamma = gamma,                            
                            reg_alpha = reg_alpha,
                            #objective='reg:squarederror'
                          )
    
    scores = []
        
    for train_index, test_index in kf.split(X, y):
        #
        X_train = scaler.transform( X[train_index] )
        y_train = y[train_index]
        #
        X_test  = scaler.transform( X[test_index] )
        y_test  = y[test_index]
        #
        clf.fit(X_train,y_train)
        #
        y_pred = clf.predict(X_test)
        #
        scores.append((rmspe(y_test,y_pred)))
                
    #
    return np.mean(np.array(scores))

optunaの試行数はtimeoutで時間で設定

In [ ]:
optuna.logging.disable_default_handler() # Optunaの出力を抑制する
#optuna.logging.enable_default_handler() # Optunaで出力する
#
n_trials = 5
#
# optuna
study = optuna.create_study()
#
study.optimize(objective_xgb, n_trials=n_trials)
#study.optimize(objective_xgb, timeout=60*2)
#study.optimize(objective_xgb, timeout=3600*5)

In [ ]:
print('best_params')
print(study.best_params)
print('best_value')
print(study.best_value)
print('best_trial')
print(study.best_trial)

In [ ]:
study_score = np.array( [ x.value for x in study.trials[:] ] )

plt.figure(figsize=(30,6))
plt.plot( study_score, 'o' )
plt.yscale('log')
plt.plot([0,len(study_score)],[study_score[0],study_score[0]],'k-')
plt.show()

plt.figure(figsize=(30,6))
plt.plot( np.sort(study_score), 'x-' )
plt.yscale('log')
plt.plot([0,len(study_score)],[study_score[0],study_score[0]],'k-')
plt.show()

In [ ]:
df_study = study.trials_dataframe()
df_study.to_csv("optuna.csv")
df_study

In [ ]:
sort = np.argsort( np.array( df_study['value'] ) )
df_study.sort_values('value')

In [ ]:
for col in df_study.columns[5:-1]:
    print(col)
    plt.figure(figsize=(30,6))
    plt.plot(df_study[col],'o')
    plt.show()
    plt.figure(figsize=(30,6))
    plt.plot(np.array(df_study.sort_values('value')[col]),'o')
    plt.show()

In [ ]:
import pickle

clf = xgb.XGBRegressor(**study.best_params, 
                       #objective='reg:squarederror'
                      )

with open('model.pickle', mode='wb') as fp:
    pickle.dump(clf, fp)
print(clf)

In [ ]:
clf.fit(scaler.transform(X),y)
y_pred = clf.predict(scaler.transform(X))

plt.figure(figsize=(4,4))    
plt.plot(y_pred,y,'x')
#
print(np.mean(np.array((rmspe(y,y_pred)))))
#
min = np.min( np.array( np.min(y_test), np.min(y_pred) ) )
max = np.max( np.array( np.max(y_test), np.max(y_pred) ) )
#
plt.plot( [0.9*min,1.1*max],[0.9*min,1.1*max], 'k-' )    
#
plt.show()

In [ ]:
plt.barh(train_info.columns[3:][::-1], clf.feature_importances_[::-1])

In [ ]:
X_test = np.array( test_info.iloc[:,3:] )

y_pred = clf.predict(scaler.transform(X_test))
y_pred

sub = test.copy()
sub['target'] = y_pred
sub = sub.drop("stock_id",axis=1)
sub = sub.drop("time_id",axis=1)
sub

In [ ]:
sub.to_csv("submission.csv", index=False)